In [10]:
import numpy as np
import time
import os
import urllib.request
from six.moves import cPickle as pickle


import tensorflow as tf
from tensorflow.models.rnn.ptb import reader
from gensim import corpora

import matplotlib.pyplot as plt



In [22]:

"""
    BEFORE RUNNING THIS CODE YOU SHOULD FIRST RUN THE "DATA_BUILDER.PY" TO FIRST EXTRACT, CLEAN AND LOAD THE DATA
    INTO PICKLE FILES AND THEN THIS CODE WILL PART COME IN HANDY.
"""

############################  NOTES:: ###########################
"""
    1. ENBEDDING(INPUT) LAYER OPERATION
       --> embed_to_hid_wghts = tf.get_variable('embedding_matrix', [vocab_size, num_hid_units]):
           embed_to_hid_layer = tf.nn.embedding_lookup(embed_to_hid_wghts, x)
           # Normally we convert the input vector into a one hot matrix and then multiply it to the embedded weights, 
           When we do so, we get the same embed weight corresponding to 1's in the one-hot vector but in a different 
           shape. The above operation does all that in a single shot. Basically, embed_to_hid_wghts defines a matrix 
           with weights going form all vacab to hiddenunits,and embed_to_hid_layer pulls the vectors from embedding_matrix
           (embed_to_hid_wghts) corresponding to the idices entries of x for all the batch. 
           So the matrix embed_to_hid_layer = [batch_size x num_sequences x num_hid_units]

    2. HIDDEN LAYER OPERATION
       --> The output from dynamic_rnn "rnn_output" is a Tensor of shape of [Batch_size x num_sequence x num_hid_units] and,
           The hid_to_output_wght is in the shape of [num_hid_units x num_classes]
           And We want an output with shape [Batch_size x num_sequence x num_classes]
           We horizontlly stack all the batches to form a matrix of [(Batch_size x num_sequence]) x num_classes]
       --> In the dynamic_run we provide the "sequence_length", this would say the RNN that the batch are padded after
           after the given size. Therefore the RNN doesnt consider the padded sequences while calculating the RNN output.
           When the actual sequence length is given then the RNN would simply consider the rnn_output as 0 for the padded
           sequence

    3. OUTPUT LAYER OPERATION
       --> sparse_softmax_cross_entropy_with_logits automatically converts the y's into on hot vectors and perform 
           the softmax operation When using softmax_cross_entropy_with_logits, we have to first convert the y's 
           into one-hot vector

    4. MASK THE LOSES:
       --> We can calculate the loss directly as we do for every batch. Normally to calculate the loss we do:
           loss_CE = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(hid_to_ouptut_layer, tf.reshape(y, [-1])))
           But here we dont do the complete stuff at once because we have zero (0) padded all the sequences in a batch for 
           equal size . Naively calculating the loss at each time step doesn’t work because that would take into account
           the padded positions. So the solution is to create a weight matrix that “masks out” the losses at padded positions.
       --> Intuition for why do we do mask. "softmax_opt" will give a array of size [(num_sequence*batch_size) * 0],
           and reduced_mean just sums up all the element of the softmax and divides it with the array size.
           And we know that many of the sequences here are padded with 0's whose softmax output should not be considered
           while performing the reduced_mean. Hence masking converts the softmax of those padded 0's to 0 and calculate
           the mean of all training example in a batch separetely. And finally computes the reduced mean. 
       
       
    5. VERY IMPORTANT NOTE:
       --> After we receive the dictionary from the gensim dictionary we need to add 1. Because the gensim dictionary 
           is build from index 0 to n. But our training or testing batch consist of index from 1 to n+1 (achieved in data_builder.py)
           . The addition of 1 is imperative because the embedding_matrix internally builds a corpus of (n*num_hidden_unit)
           and it builds from 0 so if we dont add 1 then the last word in the dictionarry will have a nan corresponding to its
           output and our loss will be nan too. Just to recap, we add 1 because we do zero (0) padding of the sequence length for a batch. 
        
    
"""

################  Some interesting facts we observe: ###############
"""
1. We apply both multilayer and single layer approach with RNN. We found that using all variable
   num_hid_units. Single layer gave better accuracy that multilayer.

"""

In [5]:
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [21]:
def dynamic_RNN_model(
    num_hid_units = 3,
    vocab_size = 7,
    momentum = 0.9,
    learning_rate = 0.1,
    num_layers = None
    ):
    print ('The num of hidden unit is: ', num_hid_units)
    print ('The Vocab size is: ', vocab_size)
    print ('The momentum is: ', momentum)
    print ('The learning_rate is: ', learning_rate)
    
    
    num_classes = vocab_size

    reset_graph()
    
    x = tf.placeholder(tf.int32, shape = [None, None], name='input_placeholder')
    y = tf.placeholder(tf.int32, shape = [None, None], name='output_placeholdr')
    x_lenarr = tf.placeholder(tf.float32, shape = [None], name='output_placeholdr')
    batch_size = tf.shape(x)[0]
    

    # ENBEDDING(INPUT) LAYER OPERATION
    # Creating an Embedding matrix with a random weight for all vacab to hidden_matrix
    embed_to_hid_wghts = tf.get_variable('embedding_matrix', [vocab_size, num_hid_units])
    embed_to_hid_layer = tf.nn.embedding_lookup(embed_to_hid_wghts, x)
    print ('The shape of embed_to_hid_wghts is: ', embed_to_hid_wghts.get_shape())
    print ('The shape of embed_to_hid_layer is: ', embed_to_hid_layer.get_shape())


    # HIDDEN LAYER OPERATION
    rnn_cell = tf.nn.rnn_cell.LSTMCell(num_hid_units, state_is_tuple=True)
    if num_layers:
        rnn_cell = tf.nn.rnn_cell.MultiRNNCell([rnn_cell] * num_layers, state_is_tuple=True)
    init_state = rnn_cell.zero_state(batch_size, tf.float32)  # Each sequence will hava a state that it passes to its next sequence
    rnn_outputs, new_state = tf.nn.dynamic_rnn(cell=rnn_cell,
                                               sequence_length=x_lenarr,
                                               initial_state=init_state,
                                               inputs=embed_to_hid_layer,
                                               dtype=tf.float32)
    

    # OUTPUT LAYER OPERATION
    # Initialize the weight and biases for the output layer. We use variable scope because we would like to share the weights 
    with tf.variable_scope('output_layer'):
        hid_to_output_wght = tf.get_variable('hid_to_output_wght', [num_hid_units, num_classes],
                                            initializer = tf.random_normal_initializer())
        output_bias = tf.get_variable('output_bias', [num_classes],
                                      initializer=tf.random_normal_initializer())
    
    rnn_outputs = tf.reshape(rnn_outputs, [-1, num_hid_units])
    hid_to_ouptut_layer = tf.matmul(rnn_outputs, hid_to_output_wght) +  output_bias  
    # Also use tf.batch_matmul(rnn_outputs, hid_to_output_wght) +  output_bias  
    output_state = tf.nn.softmax(hid_to_ouptut_layer, name=None)
 
    
    # SOFTMAX OUTPUT
    y_reshaped = tf.reshape(y, [-1])
    softmax_opt = tf.nn.sparse_softmax_cross_entropy_with_logits(hid_to_ouptut_layer, y_reshaped)
    
    # MASK THE LOSES
    mask = tf.sign(tf.to_float(y_reshaped))
    masked_loss = mask * softmax_opt
    masked_loss = tf.reshape(masked_loss,  tf.shape(y))
    mean_loss_by_example = tf.reduce_sum(masked_loss, reduction_indices=1) / x_lenarr
    mean_loss = tf.reduce_mean(mean_loss_by_example)

    
    # OPTIMIZING THE LOSS FUNCTION
#     optimizer = tf.train.AdamOptimizer(learning_rate).minimize(mean_loss)
    optimizer = tf.train.MomentumOptimizer(learning_rate, 
                                            momentum, 
                                            use_locking=False, 
                                            name='Momentum', 
                                            use_nesterov=True).minimize(mean_loss)

    # Returns graph objects
    return dict(
        x=x,
        y=y,
        x_lenarr=x_lenarr,
        batch_size = batch_size,
        init_state = init_state,
        new_state = new_state,
        loss = mean_loss,
        optimizer = optimizer,
        prediction = output_state
    )



In [24]:
class Train():
    def __init__(self, valid_num_batches=10):
        self.train_batch_dir = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Word-Search-NNets/Word-Nets/training_batch/'
        self.valid_batch_dir = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Word-Search-NNets/Word-Nets/crossvalid_batch/'
        dictionary_dir = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Word-Search-NNets/Word-Nets/dictionary.txt'
        self.vocab_size = len(corpora.Dictionary.load_from_text(dictionary_dir))
        
        # Randomly select group of 5 batches from the cross valid dataset to test after every 20 batches
        self.cdoc = np.random.choice(np.arange(30), valid_num_batches) 

    def accuracy(self, predictions, labels, labels_one_hot = None):
        # The input labels are a One-Hot Vector
        if labels_one_hot:
            return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
                  / predictions.shape[0])
        else:
            return (100.0 * np.sum(np.argmax(predictions, 1) == np.reshape(labels, [-1]))
                  / predictions.shape[0])
        
    def cross_valid(self, sess, graph_dict):
        new_valid_state_ = None
        for cdoc_no in self.cdoc:
            with open(self.valid_batch_dir+'batch'+str(cdoc_no)+'.pickle', 'rb') as f1:
                dataset = pickle.load(f1)

                batch_valid_dataset = dataset['batch_valid_dataset']
                batch_valid_labels = dataset['batch_valid_labels']
                batch_valid_lenarr = dataset['batch_valid_lenarr']

                if new_valid_state_ is not None:
                    feed_dict={graph_dict['x']: batch_valid_dataset,
                               graph_dict['x_lenarr']: batch_valid_lenarr,
                               graph_dict['init_state']: new_valid_state_}
                else:
                    feed_dict={graph_dict['x']: batch_valid_dataset,
                               graph_dict['x_lenarr']: batch_valid_lenarr}

                valid_prediction_, new_valid_state_ = sess.run([graph_dict['prediction'],
                                                                graph_dict['new_state']], 
                                                                feed_dict)

            acc_valid = self.accuracy(valid_prediction_, batch_valid_labels)
            print ('accuracy of the validation batch %d is: '%cdoc_no, acc_valid)
                        
                        
    def train_network(self, graph_dict, num_batches, 
                      when_valid, epochs=1, verbose=None, 
                      path_save_model=False):
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            for epoch in np.arange(epochs):
                new_state_ = None
                training_loss = 0
                print ('')
                print ('training training training, training training training, training training training training training')
                for no in np.arange(num_batches):
                    with open(self.train_batch_dir+'batch'+str(no)+'.pickle', 'rb') as f:
                        dataset = pickle.load(f)
                        
                        batch_train_dataset = dataset['batch_train_dataset']
                        batch_train_labels = dataset['batch_train_labels']
                        batch_train_lenarr = dataset['batch_train_lenarr']
                        
                        feed_dict= {graph_dict['x']: batch_train_dataset, 
                                    graph_dict['y']: batch_train_labels,
                                    graph_dict['x_lenarr']: batch_train_lenarr}
            
                        if new_state_ is not None:
                            feed_dict[graph_dict['init_state']] = new_state_

                        new_state_, loss_, opt, tp = sess.run([graph_dict['new_state'],
                                                            graph_dict['loss'],
                                                            graph_dict['optimizer'],
                                                            graph_dict['prediction']], 
                                                            feed_dict=feed_dict)
                        
                        training_loss += loss_
                        acc = self.accuracy(tp, batch_train_labels)
                    
                        
                        print ('accuracy of the training batch %d is: '%no, acc)
                        
                        
                        if (no%when_valid==0 and no!=0):
                            print ('Average Loss till the training batch %d is: '%no, training_loss/no)
                            print ('')
                            print ('crossvalid crossvalid crossvalid, crossvalid crossvalid crossvalid, crossvalid crossvalid crossvalid')
                            self.cross_valid(sess,graph_dict)
                            print ('')
                            print ('training training training, training training training, training training training training training')
                            
                print ('All %d Batches Done.................................'%num_batches)
                print ('')
                print ('')
                
                if verbose:
                    print("Average training loss for Epoch", epoch, ":", training_loss/num_batches)
            
            if isinstance(path_save_model, str):
                graph_dict['saver'].save(sess, path_save)
#                     




In [25]:
path_save_model="/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Word-Search-NNets/Word-Nets/singlelayerLSTM_300"
obj_Train = Train()
graph_dict =  dynamic_RNN_model(num_hid_units=300, 
                                vocab_size = obj_Train.vocab_size+1,
                                momentum = 0.9,
                                learning_rate=0.1,
                                num_layers = None)
obj_Train.train_network(graph_dict, num_batches = 10, 
                        when_valid=2, epochs=1, verbose=True, path_save_model=path_save_model)

The num of hidden unit is:  300
The Vocab size is:  17156
The momentum is:  0.9
The learning_rate is:  0.1
The shape of embed_to_hid_wghts is:  (17156, 300)
The shape of embed_to_hid_layer is:  (?, ?, 300)

training training training, training training training, training training training training training
accuracy of the training batch 0 is:  0.0
accuracy of the training batch 1 is:  1.37867647059
accuracy of the training batch 2 is:  1.99854651163
Average Loss till the training batch 2 is:  14.5978646278

crossvalid crossvalid crossvalid, crossvalid crossvalid crossvalid, crossvalid crossvalid crossvalid
accuracy of the validation batch 24 is:  1.67763157895
accuracy of the validation batch 2 is:  3.57142857143
accuracy of the validation batch 13 is:  3.18509615385
accuracy of the validation batch 20 is:  2.52976190476
accuracy of the validation batch 27 is:  2.78846153846
accuracy of the validation batch 9 is:  2.26362179487
accuracy of the validation batch 9 is:  2.28365384615
accu

KeyError: 'saver'

In [19]:
obj_Train = Train()
graph_dict =  dynamic_RNN_model(num_hid_units=300, 
                                vocab_size = obj_Train.vocab_size+1,
                                momentum = 0.9,
                                learning_rate=0.5,
                                num_layers=3)
obj_Train.train_network(graph_dict, num_batches = 50, when_valid=10, epochs=1, verbose=True)

The num of hidden unit is:  300
The Vocab size is:  17156
The momentum is:  0.9
The learning_rate is:  0.1
The shape of embed_to_hid_wghts is:  (17156, 300)
The shape of embed_to_hid_layer is:  (?, ?, 300)

training training training, training training training, training training training training training
accuracy of the training batch 0 is:  0.0
accuracy of the training batch 1 is:  0.761959876543
accuracy of the training batch 2 is:  1.01902173913
accuracy of the training batch 3 is:  2.14460784314
accuracy of the training batch 4 is:  1.00070224719
accuracy of the training batch 5 is:  0.347222222222
accuracy of the training batch 6 is:  0.696790540541
accuracy of the training batch 7 is:  0.457974137931
accuracy of the training batch 8 is:  0.475543478261
accuracy of the training batch 9 is:  2.14213709677
accuracy of the training batch 10 is:  0.0
Average Loss till the training batch 10 is:  14.1746023178

crossvalid crossvalid crossvalid, crossvalid crossvalid crossvalid, crossv